In [42]:
from get_page import simple_get
import requests, json
from bs4 import BeautifulSoup
import re
import json
import os

In [43]:
s = requests.Session()

params = {
"email":"kylesobus@gmail.com",
"password":"Sermons",
"login":"true",
"action":"login"
}

login_url = "https://www.preachingtoday.com/system/social/ajax/login.html"
s.post(login_url, params=params)

<Response [200]>

In [65]:
def get_sermon(url):
    url = 'https://www.preachingtoday.com'+url
    page = s.get(url);
    html = BeautifulSoup(page.content, 'html.parser')
    body = html.find("div", {"id": "body"})
    sermon = body.find("div", {"id":"articleBody"}).findAll("p")
    sermon_text = ""
    for p in sermon:
        sermon_text+= p.text.strip()
    summary = ''
    if body.find("p", {"class":"deck"}) is not None:
        summary = body.find("p", {"class":"deck"}).text
    doc = {
        "title":body.find("h3").text,
        "summary":summary,
        "author":body.find("div", {"class":"author-section"}).find("a").text,
        "year":url.split('/')[5],
        "month":url.split('/')[6],
        "text": sermon_text,
    }
    filename = doc['title'].replace(" ", "_") + '.json'
    filename = filename.replace("/", "")
    doc = json.dumps(doc)
    with open(filename,'w') as f:
        f.write(doc)
    print(filename)
def get_series(url):
    url = 'https://www.preachingtoday.com'+url
    page = s.get(url)
    html = BeautifulSoup(page.content, 'html.parser')
    body = html.findAll("div", {"class": "result"})
    for div in body:
        link = div.find('h4').find('a')['href']
        get_sermon(link)

In [74]:

query = {
"sourcename":"sermons",
"type":"scripture",
"query":"Genesis",
"start":1,
}
bible = ["Genesis","Exodus","Leviticus", "Numbers","Deuteronomy", "Joshua", "Judges", "Ruth", "1 Samuel", "2 Samuel",
        "1 Kings", "2 Kings", "1 Chronicles", "2 Chronicles", "Ezra", "Nehemiah", "Esther", "Job", "Psalms",
        "Proverbs", "Ecclesiastes", "Song of Songs", "Isaiah", "Jeremiah", "Lamentations", "Ezekiel", "Daniel",
        "Hosea", "Joel", "Amos", "Obadiah", "Jonah", "Micah", "Nahum", "Habakkuk", "Zephaniah", "Haggai","Zechariah",
        "Malachi", "Matthew", "Mark", "Luke", "John", "Acts", "Romans", "1 Corinthians", "2 Corithians", "Galatians",
        "Ephesians", "Philippians", "Colossians", "1 Thessalonians", "2 Thessalonians", "1 Timothy", "2 Timothy",
        "Titus", "Philemon", "Hebrews", "James", "1 Peter", "2 Peter", "1 John", "2 John", "3 John", "Jude", "Revelation"]
total = 0
for books in bible:
    print("--------------", books, "--------------")
    start = 1
    query_url = "https://www.preachingtoday.com/search/ajax/search_response.html"
    query['query'] = books
    data = s.get(query_url, params = query)
    html = BeautifulSoup(data.content, 'html.parser')
    num_results = int(html.find("span", {"class": "review-result"}).text.split(" ")[2])
    print("Num Results: ", num_results, start)
    total += num_results
    while start < num_results:
        query_url = "https://www.preachingtoday.com/search/ajax/search_response.html"
        query['query'] = books
        data = s.get(query_url, params = query)
        html = BeautifulSoup(data.content, 'html.parser')
        header = html.findAll('h3')
        for h in header:
            link = h.find("a")
            if link is not None:
                if link['href'].find('sermon-series') > -1:
                    get_series(link['href'])
                else:
                    get_sermon(link['href'])
        if start+20 < num_results:
            start +=20
        elif start == num_results:
            start = num_results +1
        else:
            start = num_results
        query['start'] = start
print("Total number of sermons scraped: ", total)

-------------- 2 Peter --------------
Num Results:  9 1
Unprovoked_Love.json
The_Seven_Habits_of_Highly_Effective_Believers.json
Inspiration.json
Canonization.json
Illumination.json
Meditation.json
Application.json
Inspiration.json
Is_Our_Church_a_Healthy_Church?.json
What_About_That_Other_Bible.json
Wholesome_Thinking.json
The_End_of_the_World_as_We_Know_It.json
Captured_by_the_Word.json
Total number of sermons scraped:  9


In [46]:
query = {
"sourcename":"sermons",
"type":"scripture",
"query":"Mark",
"start":1,
}
query_url = "https://www.preachingtoday.com/search/ajax/search_response.html"
data = s.get(query_url, params = query)

In [75]:
count = 0
for doc in os.listdir('./preachingtoday'):
    count +=1
count

1267

In [70]:
os.getcwd()

'/Users/zdunkerton/Documents/data/machine_churching'

In [69]:
os.chdir('..')